# Inleiding MongoDB

* **Voorkennis:** [Inleiding Jupyter](Inleiding-Jupyter.ipynb)

In dit notebook geven we een aantal voorbeelden van MongoDB, aan de hand van de MongoDB shell (de commando-regel).
Hiermee kun je oefenen met zoekopdrachten e.d.
Het gebruik van een MongoDB-database in een Python-toepassing hebben we uitgewerkt in de andere Notebooks.
Meer informatie over de MongoDB shell-opdrachten vind je via: [Mongo shell](https://docs.mongodb.com/manual/mongo/).

In dit notebook moet je per cell de mongo-shell opstarten; hiervoor gebruiken je de opdracht `mongo demo --quiet`.
Daarna kun je één of meer mongo-shell opdrachten geven.

De onderstaande opdracht is nodig als initialisatie in sommige omgevingen.

In [ ]:
import os
os.environ["PATH"] += ":/usr/local/bin"

## Importeren voorbeeld-database

Als eerste opdracht lezen we de voorbeeld-database in, met `mongoimport`. Dit hoeft je maar één keer per sessie te doen. Je kunt hiermee ook de database herstellen als je deze verziekt hebt.

In de onderstaande opdracht importeren we de *collection* `contacts` uit het bestand `adressen.json`.

In [ ]:
%%bash
mongoimport -d demo --drop -c contacts adressen.json

Met de onderstaande opdracht bekijk je het bestand `adressen.json`.

**Ga na**:

* welke documenten hebben geen email-adres?
* welke documenten hebben meerdere email-adressen?
* welke documenten hebben geen adres?
* welke documenten hebben geen postcode in het adres?


In [ ]:
%%bash
cat adressen.json

## Eerste zoekopdrachten

De zoekopdracht in de mongo-shell heeft de vorm: `db.collection.find(query-doc)`. Hierin is `query-doc` het zoek- document waarin een deel van de velden ingevuld is. De zoekopdracht vindt alle documenten die matchen met dit zoek-document.

Als je geen zoekdocument opgeeft, of een leeg zoekdocument (`{}`), dan krijg je alle documenten in de collection.

In [ ]:
%%bash
mongo demo --quiet

db.contacts.find({})

### Key: `_id`-veld

Merk op dat de geïmporteerde documenten een veld `_id` hebben: dit is de unieke key van het document die MongoDB automatisch aangemaakt heeft. Deze key is uniek voor alle databases en collections.

### Zoeken naar enkel document

Door een veld op te geven dat uniek is binnen de collectie, vind je een enkel document.

In [ ]:
%%bash
mongo demo --quiet

db.contacts.find({"name": "Hans de Boer"})

**Opdracht** zoek een document op basis van het email-veld.

In [ ]:
%%bash
mongo demo --quiet

db.contacts.find({???})

**Opdracht** zoek een document op basis van het `_id`-veld.

In [ ]:
%%bash
mongo demo --quiet

db.contacts.find({???})

## Projectie

Met "find" selecteren we een verzameling documenten. De algemene vorm is: `db.collection.find({<selection>}, {<projection>})`. Met het projectie-document geef je aan welke velden van deze documenten je in het resultaat wilt zien: een veld in het resultaat geeft je aan met `fieldname:1`; het weglaten van een veld geef je aan met `fieldname:0`. Afgezien van het `_id`-field, kun je in een projectie-document alleen "0"-en of "1"-en opgeven.

> In een SQL-querie geef je na de `SELECT` een opsomming van de kolommen in het resultaat: dit is de SQL-vorm van projectie.

**Voorbeeld**

In [ ]:
%%bash
mongo demo --quiet

db.contacts.find({"name": "Hans de Boer"}, {"_id":0, "name":1, "email": 1})

**Opdracht.** De volgorde van de velden in het resultaat hangt af van de oorspronkelijke definitie van het document, niet van de volgorde van de velden in de projectie. **Ga dit na**, door onderstaande query aan te passen.

> Dit is een verschil met SQL: daar bepaalt de query de volgorde van de kolommen in het resultaat.

In [ ]:
%%bash
mongo demo --quiet

db.contacts.find({"name": "Hans de Boer"}, {"_id":0, "name":1, "email": 1})

## Selectie in een samengestelde waarde

Het adres-veld is een samengestelde waarde, met de velden `address.street`, `address.city` en `address.poscode`. Ook in zo'n samengestelde waarde kun je zoeken:

In [ ]:
%%bash
mongo demo --quiet

db.contacts.find({"address.city": "Amsterdam"})

**Opdracht** Dit heeft een andere betekenis dan `{"address":{"city": "Amsterdam"}}` . **Ga dit na**. (Waarom zou dat zo zijn?)

In [ ]:
%%bash
mongo demo --quiet

db.contacts.find({???}})

**Opdracht.** Je kunt selectie en projectie combineren. 
Maak een query met als resultaat (alleen) de namen van alle contacten met woonplaats Amsterdam.

In [ ]:
%%bash
mongo demo --quiet

db.contacts.find({<selectie>}, {<projectie>})

## Selectie en meervoudige waarden

In de voorbeeld-documenten komen documenten voor met meervoudige waarden voor het `email`-veld. Met een enkelvoudig veld in de zoekopdracht vind je ook docementen met zo'n samengestelde waarde:

In [ ]:
%%bash
mongo demo --quiet

db.contacts.find({"email": "leontien134@tiscalimail.nl"})

## And en or

Als je in het query-document meerdere velden invult, dan voldoen de zoekresultaten aan de "en" (*and*) van deze ingevulde velden.

Een *or* is wat ingewikkelder: je gebruikt daarvoor een `$or`, met een lijst van alternatieven. Voorbeeld:

In [ ]:
%%bash
mongo demo --quiet

db.contacts.find({"$or": [{"address.city": "Amsterdam"}, {"address.city": "Rotterdam"} ]})

## Ontbrekende velden

Zoals we gezien hebben kunnen de documenten in een collection nogal verschillen in de velden die ze definiëren.
Met de query `{field: {$exists: false}}` vind je de documenten zonder `field`.

In [ ]:
%%bash
mongo demo --quiet

db.contacts.find({"address.city": {"$exists": false}})

**Opdracht** vind de documenten zonder postcode.

In [ ]:
%%bash
mongo demo --quiet

db.contacts.find({???})

**Opdracht.** vind de documenten **met** postcode. (Wat is het alternatief voor `false`?)

In [ ]:
%%bash
mongo demo --quiet

db.contacts.find({???})

## Reguliere expressies

In de query kun je ook een *reguliere expressie* opgeven, om op een ruimere manier strings te matchen.
Zie voor de uitgebreide mogelijkheden de MongoDB-documentatie.

Een voorbeeld hoe je dit in MongoDB gebruikt: zoek alle documenten waarvan de `address.city`-string de substring `dam` bevat. (In dit geval, "Amsterdam" en "Rotterdam").

In [ ]:
%%bash
mongo demo --quiet

db.contacts.find({"address.city": {"$regex": /dam/}})

**Opdracht.** Zoek alle documenten waarvan de naam "Anna" bevat.

In [ ]:
%%bash
mongo demo --quiet

db.contacts.find({???})

## Verder

* [Inhoudsopgave](Inhoud.ipynb)